In [2]:
import functools
import requests
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import json

In [3]:
class DB():
    def __init__(self, server, timerange=7):
        self.server = server
        self.timerange = timerange

    @functools.lru_cache(maxsize=512)
    def get(self, item, timerange=None):
        time.sleep(.25) # used to rate limit the api call, sleep is skipped when cached
        url = f'https://api.nexushub.co/wow-classic/v1/items/{self.server}/{item}/prices?timerange={timerange or self.timerange}'
        return requests.get(url).json()
    
    @functools.lru_cache(maxsize=512)
    def get_df(self, item):
        df = pd.DataFrame(self.get(item)['data'])
        df['scannedAt'] = pd.to_datetime(df['scannedAt'])
        df['marketValue'] /= 100
        df['minBuyout']   /= 100
        df['minBuyoutPctile'] = df['minBuyout'].rank(pct=True)
        return df
    
    @functools.lru_cache(maxsize=512)
    def get_df_hourly(self, item):
        return self.get_df(item).resample('60min', on='scannedAt').mean().ffill().reset_index()

    @functools.lru_cache(maxsize=512)
    def get_vendorprice(self, item):
        url = f'https://api.nexushub.co/wow-classic/v1/item/{item}'
        return requests.get(url).json()['sellPrice']/100 or 0
    
    def get_name(self, item):
        return self.get(item)['name']

    def get_last_record(self, item):
        return self.get_df(item).iloc[-1]
    
    def plot(self, item, y):
        self.get_df(item).plot(x='scannedAt', y=y, label=f'{self.get_name(item)} | {y}')

In [4]:
print('''
Usage: 
  db = DB(server)          --> initializes database
  db.get_df(item)          --> returns dataframe, prices divided by 100 so copper in decimal 
  db.get_name(item)        --> returns name of item  
  db.get_last_record(item) --> returns most recent record
  db.plot(item, y)         --> plots column over time
''')


Usage: 
  db = DB(server)          --> initializes database
  db.get_df(item)          --> returns dataframe, prices divided by 100 so copper in decimal 
  db.get_name(item)        --> returns name of item  
  db.get_last_record(item) --> returns most recent record
  db.plot(item, y)         --> plots column over time

